In [11]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
cd ..

/Share/home/shibinbin/projects/emaize


In [3]:
import h5py

In [12]:
X = []
with h5py.File('data/parent_genotype_2bit/all_matrix') as f:
    X.append(f['X_female'][:])
    X.append(f['X_male'][:])
X = np.concatenate(X, axis=0)

In [24]:
random_trait = np.random.normal(size=X.shape[0])
X_small = np.take(X, np.arange(10000), axis=1)

In [14]:
X_small

array([[1, 0, 1, ..., 0, 1, 1],
       [1, 0, 1, ..., 1, 1, 0],
       [1, 0, 1, ..., 0, 1, 1],
       ..., 
       [1, 0, 1, ..., 0, 1, 0],
       [1, 0, 1, ..., 0, 1, 0],
       [1, 0, 1, ..., 0, 1, 0]], dtype=int8)

In [26]:
import scipy.stats
def fast_anova_2bit(X, y):
    y = y - y.mean()
    y2 = y*y
    N = X.shape[0]
    SS_tot = np.sum(y2)
    # 10, 01, 11
    masks = [np.logical_and(X[:, 0::2], np.logical_not(X[:, 1::2])),
             np.logical_and(np.logical_not(X[:, 0::2]), X[:, 1::2]),
             np.logical_and(X[:, 0::2], X[:, 1::2])]
    Ni = np.concatenate([np.sum(mask, axis=0) for mask in masks]).reshape((3, -1))
    at_least_one = Ni > 0
    SS_bn = [np.sum(y.reshape((-1, 1))*mask, axis=0) for mask in masks]
    SS_bn = np.concatenate(SS_bn).reshape((3, -1))
    SS_bn **= 2
    SS_bn = np.where(at_least_one, SS_bn/Ni, 0)
    SS_bn = np.sum(SS_bn, axis=0)
    SS_wn = SS_tot - SS_bn
    M = np.sum(at_least_one, axis=0)
    DF_bn = M - 1
    DF_wn = N - M
    SS_bn /= DF_bn
    SS_wn /= DF_wn
    F = SS_bn/SS_wn
    
    p_vals = np.ones(F.shape[0])
    ind = np.nonzero(M == 2)[0]
    if ind.shape[0] > 0:
        p_vals[ind] = scipy.stats.f.sf(F[ind], 1, N - 2)
    ind = np.nonzero(M == 3)[0]
    if ind.shape[0] > 0:
        p_vals[ind] = scipy.stats.f.sf(F[ind], 2, N - 3)
    return F, p_vals

F_fast, pvals_fast = fast_anova_2bit(X_small, random_trait)

/Share/home/shibinbin/apps/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:16: RuntimeWarning: invalid value encountered in divide
/Share/home/shibinbin/apps/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: RuntimeWarning: divide by zero encountered in divide


In [37]:
def anova(X, y):
    masks = [np.logical_and(X[:, 0::2], np.logical_not(X[:, 1::2])),
             np.logical_and(np.logical_not(X[:, 0::2]), X[:, 1::2]),
             np.logical_and(X[:, 0::2], X[:, 1::2])]
    N = X.shape[1]/2
    Fs = np.full(N, np.nan)
    pvals = np.full(N, np.nan)
    for i in range(X.shape[1]/2):
        samples = []
        for j in range(3):
            mask = masks[j][:, i]
            if np.sum(mask) > 0:
                samples.append(y[mask])
        if len(samples) > 1:
            Fs[i], pvals[i] = scipy.stats.f_oneway(*samples)
    return Fs, pvals
F, pvals = anova(X_small, random_trait)

In [38]:
not_nan_mask = ~np.isnan(F)
np.allclose(pvals_fast[not_nan_mask], pvals[not_nan_mask])

True

In [39]:
not_nan_mask = ~np.isnan(F)
np.allclose(F_fast[not_nan_mask], F[not_nan_mask])

True